# Set LLM API

In [ ]:
!pip install langchain
!pip install openai

In [ ]:
!pip install pipreqs
!pip install pipreqsnb

In [ ]:
# !cp /finetuning_Mixtral.py /content
# !pipreqs /content --force

In [3]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
import pandas as pd
import openai
import os
from difflib import SequenceMatcher

In [ ]:
# put the huggingface api key

openai_key=""
os.environ['OPENAI_API_KEY'] = openai_key
openai.api_key = openai_key

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0.2, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

DataFrame

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


df_sentences

df_paragraphs

df_output

In [55]:
# Load the Excel file
xlsx_file = pd.ExcelFile('/content/Online124.xlsx')

# Get the sheet names
sheet_names = xlsx_file.sheet_names

# Create dataframes with specific names
df_sentences = xlsx_file.parse(sheet_names[0])  # Assuming the first sheet contains sentences
df_paragraphs = xlsx_file.parse(sheet_names[1])  # Assuming the second sheet contains paragraphs

# Now you have two dataframes named df_sentences and df_paragraphs

In [40]:
df_sentences

,ID,Sentence,Paragraph-id,Satisfied Requirement-1,Matching Degree-1,Satisfied Requirement-2,Matching Degree-2,Satisfied Requirement-3,Matching Degree-3,Comments,...,Unnamed: 78,Unnamed: 79,Unnamed: 80,Unnamed: 81,Unnamed: 82,Unnamed: 83,Unnamed: 84,Unnamed: 85,Unnamed: 86,Unnamed: 87
0,1,Data Processor Agreement,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Appendix A,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Data Processor Agreement,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,Introduction,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,This agreement re processing of personal data ...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,110,Supplier of data in case of add-on agreement: ...,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,111,"MailJet, 13-13 bis Rue de l’Aubrac - 75012 Par...",14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
109,112,New Sub-Processors,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110,113,The following Sub-Processors have been added a...,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_sentences.columns

In [62]:
df_paragraphs.columns

Index(['paragraph-id', 'paragraph'], dtype='object')

In [56]:
# Function to extract the requirement code from a given string or integer
def extract_requirement_code(s):
    if pd.isna(s):
        return ''  # Return 'R99' for NaN values
    if isinstance(s, float) or isinstance(s, int):
        return f'R{s}'  # Convert integer to string with 'R' prefix
    return s.split(' ')[0]  # Split the string and return the first part

# Apply this function to the relevant columns
for col in ['Satisfied Requirement-1', 'Satisfied Requirement-2', 'Satisfied Requirement-3']:
    df_sentences[col] = df_sentences[col].apply(extract_requirement_code)

# Assuming the labels are R1 through R47
all_labels = [f'R{i}' for i in range(1, 47)]

# Initialize a dictionary to hold binary vectors for each label
label_vectors = {label: [] for label in all_labels}

# Iterate over each row in the dataset and update the binary vectors
for _, row in df_sentences.iterrows():
    current_labels = [row['Satisfied Requirement-1'], row['Satisfied Requirement-2'], row['Satisfied Requirement-3']]
    for label in all_labels:
        # If the label is in the current row's labels, append 1, otherwise 0
        label_vectors[label].append(1 if label in current_labels else 0)

# Convert the dictionary to a DataFrame
label_df = pd.DataFrame(label_vectors)

# Combine the original data with the new label DataFrame
df_sentences = pd.concat([df_sentences, label_df], axis=1)

def format_gdpr_requirements(row):
    labels = [label for label in all_labels if row.get(label, 0) == 1]
    return ','.join(labels) if labels else 'R99'

# Apply the format_gdpr_requirements function only to label columns to create the 'Target' column
df_sentences['Target'] = df_sentences[all_labels].apply(format_gdpr_requirements, axis=1)


In [57]:
df_sentences

,ID,Sentence,Paragraph-id,Satisfied Requirement-1,Matching Degree-1,Satisfied Requirement-2,Matching Degree-2,Satisfied Requirement-3,Matching Degree-3,Comments,...,R38,R39,R40,R41,R42,R43,R44,R45,R46,Target
0,1,Data Processor Agreement,1,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
1,2,Appendix A,1,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
2,3,Data Processor Agreement,1,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
3,4,Introduction,1,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
4,5,This agreement re processing of personal data ...,1,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,110,Supplier of data in case of add-on agreement: ...,14,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
108,111,"MailJet, 13-13 bis Rue de l’Aubrac - 75012 Par...",14,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
109,112,New Sub-Processors,15,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99
110,113,The following Sub-Processors have been added a...,15,,NaN,,NaN,,NaN,NaN,...,0,0,0,0,0,0,0,0,0,R99


In [29]:
len(df_sentences['Target'])

112

In [58]:
df_sentences.iloc[104]

ID                                             107
Sentence                   Approved Sub-Processors
Paragraph-id                                    14
Satisfied Requirement-1                           
Matching Degree-1                              NaN
                                    ...           
R43                                              0
R44                                              0
R45                                              0
R46                                              0
Target                                         R99
Name: 104, Length: 135, dtype: object

In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for statement in df_sentences['Sentence']:
    print(statement)
    print('++++++')

In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for paragraph_id in df_sentences['Paragraph-id']:
    print(paragraph_id)
    print('++++++')

In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for target in df_sentences['Target']:
    if not target.strip():
        print('None')
    else:
        print(target)
    print('++++++')

In [ ]:
prompt_template = """ You are a legal expert trained to identify applicable {{policies}} based on a given {{text}}.
Your response should only include the policy identifier (e.g., 'R5') if applicable. If there is no direct connection to any policy, respond with '0'.
Do not include any explanations or additional text.
For example:
- If a text matches a policy, respond with 'R[number]'.
- If a text does not match any policy, respond with '0'.
Follow this format strictly.

Policies: {{R1 - The DPA shall contain at least one controller's identity and contact details.  (LL)
R2 - The DPA shall contain at least one processor's identity and contact details. (LL)
R3 - The DPA shall contain the duration of the processing.  (Art. 28(3))
R4 - The DPA shall contain the nature and purpose of the processing. (Art. 28(3))
R5 - The DPA shall contain the types of personal data. (Art. 28(3))
R6 - The DPA shall contain the categories of data subjects.  (Art. 28(3))
R7 - The  organizational and technical measures to ensure a level of security can include: (a) pseudonymisation and encryption of personal data, (b) ensure confidentiality, integrity, availability and resilience of processing systems and services, (c) restore the availability and access to personal data in a timely manner in the event of a physical or technical incident, and (d) regularly testing, assessing and evaluating the effectiveness of technical and organisational measures for ensuring the security of the processing.  (Art. 32(1))
R8 - The notification of personal data breach shall at least include (a) the nature of personal data breach; (b) the name and contact details of the data protection officer; (c) the consequences of the breach; (d) the measures taken or proposed to mitigate its effects. (Art. 33(3))
R9 - The DPIA shall at least include (a) a systematic description of the envisaged processing operations and the purposes of the processing, (b) an assessment of the necessity and proportionality of the processing operations in relation to the purposes, (c) an assessment of the risks to the rights and freedoms of data subjects, and (d) the measures envisaged to address the risks.  (Art. 35(7))
R10 - The processor shall not engage a sub-processor without a prior specific or general written authorization of the controller.  (Art. 28(2))
R11 - In case of general written authorization, the processor shall inform the controller of any intended changes concerning the addition or replacement of sub-processors. (Art. 28(2))
R12 - The processor shall process personal data on documented instructions from the controller.  (Art. 28(3a))
R13 - The processor can process personal data without documented instructions, if required by Union or Member State law.  (Art. 28(3a))
R14 - The processor shall inform the controller of that legal requirement before processing, if law does not prohibit informing the controller on grounds of public interest. (Art. 28(3a))
R15 - The processor shall ensure that persons authorized to process personal data have committed themselves to confidentiality or are under an appropriate statutory obligation of confidentiality.  (Art. 28(3b))
R16 - The processor shall take all measures required pursuant to Article 32 or to ensure the security of processing.  (Art. 28(3c))
R17 - The processor shall assist the controller in fulfilling its obligation to respond to requests for exercising the data subject's rights.  (Art. 28(3e))
R18 - The processor shall assist the controller in ensuring the security of processing. (Art. 28(3f), Art. 32)
R19 - The processor shall assist the controller in consulting the supervisory authorities prior to processing where the processing would result in a high risk in the absence of measures taken by the controller to mitigate the risk.  (Art. 28(3f), Art.36)
R20 - The processor shall assist the controller in notifying a personal data breach to the supervisory authority.  (Art. 28(3f), Art.33)
R21 - The processor shall assist the controller in communicating a personal data breach to the data subject. (Art. 28(3f), Art.34)
R22 - The processor shall assist the controller in ensuring compliance with the obligations pursuant to data protection impact assessment (DPIA). (Art. 28(3f), Art.35)
R23 - The processor shall return or delete all personal data to the controller after the end of the provision of services relating to processing.  (Art. 28(3g))
R24 - The processor shall immediately inform the controller if an instruction infringes the GDPR or other data protection provisions.  (Art. 28(3h))
R25 - The processor shall make available to the controller information necessary to demonstrate compliance with the obligations Article 28 in GDPR.  (Art. 28(3h))
R26 - The processor shall allow for and contribute to audits, including inspections, conducted by the controller or another auditor mandated by the controller.  (Art. 28(3h))
R27 - The processor shall impose the same obligations referred to in Article 28(3) in GDPR on the engaged sub-processors by way of contract or other legal act under Union or Member State law.   (Art. 28(4))
R28 - The processor shall remain fully liable to the controller for the performance of sub-processor's obligations.  (Art. 28(4))
R29 - When assessing the level of security, the processor shall take into account the risk of accidental or unlawful destruction, loss, alternation, unauthorized disclosure of or access to the personal data transmitted, stored or processed. (Art. 32(2))
R30 - The processor shall not transfer personal data to a third country or international organization without  a prior specific or general authorization of the controller. (Art. 28(3a))
R31 - The processor can demonstrate guarantees to Article 28 (1--4) through adherence to an approved codes of conduct or an approved certification mechanism. (Art. 28(5))
R32 - The processor shall implement appropriate technical and organisational measures to ensure a level of security appropriate to the risk of varying likelihood and severity for the rights and freedoms of natural persons.  (Art. 32(1))
R33 - The processor shall ensure that any natural person acting under its authority who has access to personal data only process them on instructions from the controller. (Art. 32(4))
R34 - The processor shall notify the controller without undue delay after becoming aware of a personal data breach.  (Art. 33(2))
R35 - A processor shall be liable for the damage caused by processing only where it has not complied with obligations of the GDPR specifically directed to processors or where it has acted outside or contrary to lawful instructions of the controller.   (Art. 82(2))
R36 - In case of general written authorization, the controller shall have the right to object to changes concerning  the addition or replacement of sub-processors, after having been informed of such intended changes by the processor.   (Art. 28(2))
R37 - The controller shall have the right to suspend the processing in certain cases. (LL)
R38 - The controller shall have the right to terminate the DPA in certain cases. (LL)
R39 - The controller shall, no later than 72 hours after having become aware of it, notify the personal data breach to the supervisory authority. (Art. 33(1))
R40 - The controller shall document the personal breaches. (Art. 33(5))
R41 - In case of high risks, the controller shall communicate the data breach to the data subject without undue delay.  (Art. 34(1))
R42 - The controller shall carry out DPIA.  (Art. 35(1))
R43 - The controller shall seek advice of the DPO when carrying DPIA. (Art. 35(2))
R44 - The controller shall seek the views of data subjects or their representatives on the intended processing. (Art. 35(9))
R45 - The controller shall carry out a review to assess if processing is performed in accordance with the data protection impact assessment at least when there is a change of the risk represented by processing operations.   (Art. 35(11)
R46 - Any controller involved in processing shall be liable for the damage caused by processing which infringes the GDPR. (Art. 82(2))}}
"""

few_shot_prompt = """

--- example start ---
Text: {{[NAME ACCOUNTANCY PRACTICE], with its registered office in [city, street and house number], hereinafter referred to as: "Processor", "We", "Us" or "Our", duly represented in this matter by [name + position];}}
Prediction: R2</stop>

Text: {{The Processing will be carried out in accordance with Your written instructions, unless We are obliged by law or regulations to act differently (for example, when considering whether an "unusual transaction" should be reported within the context of the Money Laundering and Terrorist Financing Prevention Act (Wwft)).}}
Prediction: R12</stop>

Text: {{We shall not be liable for any damage suffered as a result of Your failure to comply with the GDPR or other laws or regulations.}}
Prediction: R6</stop>

Text: {{Data Breach Notification Duty: The duty to report Data Breaches to the Dutch Data Protection Authority and (in some cases) to the Data Subject(s).}}
Prediction: 0</stop>

Text: {{It is not possible to terminate this Agreement prematurely.}}
Prediction: 0</stop>

Text: {{"Indemnification shall apply not only to the damage that Third Parties may have suffered (both material and immaterial), but also to the costs We must incur in connection therewith, for example in any legal proceedings, and to the costs of any fines imposed on Us as a result of Your acts."}}
Prediction: 0</stop>
--- example end ---
"""

example_template = "Text: {clause}\nPrediction: "

In [ ]:
def call_gpt(input, prompt_template=prompt_template, few_shot_prompt=few_shot_prompt, temperature=0.5):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "system", "content": prompt_template+few_shot_prompt},
        {"role": "user", "content": input}
    ],
    temperature=temperature
    )

    return response.choices[0].message["content"]

sentence level

In [ ]:
final_df = pd.DataFrame(columns=["paragraph_id", "sentence", "target", "prediction"])

# Indices to skip
skip_indices = [0, 1, 2, 3, 6, 11, 20, 25, 26, 30, 35, 37, 41, 49, 54, 58, 69, 75, 82, 84, 91, 93, 94, 99, 103, 104, 109]

# Process each row and save results
for index, row in df.iterrows():
    if index in skip_indices:
        continue  # Skip the specified rows
    paragraph_id, sentence, target = row['paragraph_id'], row['sentence'], row['Target']
    prediction = call_gpt(example_template.format(sentence))

    # Add the results to final_df
    final_df.loc[len(final_df)] = [paragraph_id, sentence, target, prediction]

    # Save the current state of final_df to a CSV file
    final_df.to_csv('results.csv', index=False)

# The final DataFrame is ready and saved incrementally to 'incremental_results.csv'
final_df.head()  # Display the first few rows of the final DataFrame


paragraph level

In [ ]:
!pip install tiktoken

In [61]:
import tiktoken
# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# not exact!
# simplified from https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3
    return num_tokens

def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [ ]:
def get_completion_from_messages(messages,
                                 model="ft:gpt-3.5-turbo",
                                 temperature=0.4,
                                 max_tokens=None):
    # If max_tokens is not provided, calculate it based on the message tokens
    if max_tokens is None:
        input_tokens = num_tokens_from_messages(messages)
        max_tokens = 4096 - input_tokens

    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for paragraph_id in df_paragraphs['paragraph-id']:
    print(paragraph_id)
    print('++++++')

In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for paragraph in df_paragraphs['paragraph']:
    print(paragraph)
    print('++++++')

In [ ]:
text = """1
++++++
2
++++++
3
++++++
4
++++++
5
++++++
6
++++++
7
++++++
8
++++++
9
++++++
10
++++++
11
++++++
12
++++++
13
++++++
14
++++++
15
++++++
"""

# Split the text into sections using the delimiter "++++++"
paragraph_id = text.split('++++++')

# Remove leading and trailing spaces from each section
paragraph_id = [paragraph_id.strip() for paragraph_id in paragraph_id if paragraph_id.strip()]

# Create a dataframe with a single column 'Section' using the cleaned sections
df_paragraph = pd.DataFrame({'paragraph_id': paragraph_id})
# Add the 'Section' column to the dataframe
df_paragraph['paragraph'] = paragraph

In [ ]:
prompt_template = """
You are a legal expert trained to identify applicable {{policies}}  based on a given {{text}} within its specific {{context}}.
When provided with the {{text}} and its {{context}}, your response should only include the policy identifier (e.g., 'R5') if applicable.
If there is no direct connection to any policy within the context provided, respond with '0'.
Do not include any explanations or additional text. Follow this format strictly.
"""
few_shot_prompt = """

--- example start ---
Text: {{[NAME ACCOUNTANCY PRACTICE], with its registered office in [city, street and house number], hereinafter referred to as: "Processor", "We", "Us" or "Our", duly represented in this matter by [name + position];}}
Prediction: R2</stop>

Text: {{The Processing will be carried out in accordance with Your written instructions, unless We are obliged by law or regulations to act differently (for example, when considering whether an "unusual transaction" should be reported within the context of the Money Laundering and Terrorist Financing Prevention Act (Wwft)).}}
Prediction: R12</stop>

Text: {{We shall not be liable for any damage suffered as a result of Your failure fto comply with the GDPR or other laws or regulations.}}
Prediction: R6</stop>

Text: {{Data Breach Notification Duty: The duty to report Data Breaches to the Dutch Data Protection Authority and (in some cases) to the Data Subject(s).}}
Prediction: 0</stop>

Text: {{It is not possible to terminate this Agreement prematurely.}}
Prediction: 0</stop>

Text: {{"Indemnification shall apply not only to the damage that Third Parties may have suffered (both material and immaterial), but also to the costs We must incur in connection therewith, for example in any legal proceedings, and to the costs of any fines imposed on Us as a result of Your acts."}}
Prediction: 0</stop>
--- example end ---
"""

# Function to call GPT with the sentence and its context paragraph
def call_gpt(sentence, context, model="gpt-3.5-turbo", temperature=0,max_tokens=None):
    # If max_tokens is not provided, calculate it based on the message tokens
    if max_tokens is None:
        input_tokens = num_tokens_from_messages(messages)
        max_tokens = 4096 - input_tokens

    # Call to the OpenAI API with the constructed messages
    response = openai.ChatCompletion.create(
        model=model,
    # Combine prompt template, few-shot examples, and the actual user input containing sentence and context
        messages = [
            {"role": "system", "content": prompt_template + few_shot_prompt},
            {"role": "user", "content": f"Context: {context}\nSentence: {sentence}\nPrediction:"}
        ],
        temperature=temperature,
        max_tokens=max_tokens,
    )

    return response.choices[0].message["content"]

final_df = pd.DataFrame(columns=["paragraph_id", "sentence", "target", "prediction"])

skip_indices = [0, 1, 2, 3, 6, 11, 20, 25, 26, 30, 35, 37, 41, 49, 54, 58, 69, 75, 82, 84, 91, 93, 94, 99, 103, 104, 109]

for index, row in df.iterrows():
    if index in skip_indices:
        continue  # Skip the specified rows

    paragraph_id, sentence, target = row['paragraph_id'], row['sentence'], row['Target']

    # Check if there is a matching paragraph_id in df_paragraph
    if df_paragraph[df_paragraph['paragraph_id'] == paragraph_id].empty:
        print(f"No matching paragraph for paragraph_id {paragraph_id} at index {index}")
        continue

    context = df_paragraph[df_paragraph['paragraph_id'] == paragraph_id]['paragraph'].iloc[0]

    # Call the function with the sentence and its context
    prediction = call_gpt(sentence, context)

    # Add the results to final_df
    final_df.loc[len(final_df)] = [paragraph_id, sentence, target, prediction]


In [ ]:
# Set the display option for column width to a larger value
pd.set_option('display.max_colwidth', None)

# Iterate over each row in the 'Statement' column and print the full text
for paragraph in df_paragraphs['paragraph']:
    print(paragraph)
    print('++++++')

Count tokens for cost prediction sentence-level and paragraph-level

In [69]:
import tiktoken
import numpy as np

# Token counting functions
encoding = tiktoken.get_encoding("cl100k_base")

# Counts total tokens for all messages
def num_tokens_from_messages(messages, tokens_per_message=3, tokens_per_name=1):
    num_tokens = 0
    for message in messages:
        num_tokens += tokens_per_message
        for key, value in message.items():
            num_tokens += len(encoding.encode(value))
            if key == "name":
                num_tokens += tokens_per_name
    num_tokens += 3  # Adjusted as per the OpenAI Cookbook example
    return num_tokens

# Counts tokens for assistant messages
def num_assistant_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "assistant":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

# Counts tokens for system messages
def num_system_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "system":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

# Counts tokens for user messages
def num_user_tokens_from_messages(messages):
    num_tokens = 0
    for message in messages:
        if message["role"] == "user":
            num_tokens += len(encoding.encode(message["content"]))
    return num_tokens

def print_distribution(values, name):
    print(f"\n#### Distribution of {name}:")
    print(f"min / max: {min(values)}, {max(values)}")
    print(f"mean / median: {np.mean(values)}, {np.median(values)}")
    print(f"p5 / p95: {np.quantile(values, 0.1)}, {np.quantile(values, 0.9)}")


In [66]:
compliance_rules_text = """R1 - The DPA shall contain at least one controller's identity and contact details.
R2 - The DPA shall contain at least one processor's identity and contact details.
R3 - The DPA shall contain the duration of the processing.
R4 - The DPA shall contain the nature and purpose of the processing.
R5 - The DPA shall contain the types of personal data.
R6 - The DPA shall contain the categories of data subjects.
R7 - The  organizational and technical measures to ensure a level of security can include: (a) pseudonymisation and encryption of personal data, (b) ensure confidentiality, integrity, availability and resilience of processing systems and services, (c) restore the availability and access to personal data in a timely manner in the event of a physical or technical incident, and (d) regularly testing, assessing and evaluating the effectiveness of technical and organisational measures for ensuring the security of the processing.
R8 - The notification of personal data breach shall at least include (a) the nature of personal data breach; (b) the name and contact details of the data protection officer; (c) the consequences of the breach; (d) the measures taken or proposed to mitigate its effects.
R9 - The DPIA shall at least include (a) a systematic description of the envisaged processing operations and the purposes of the processing, (b) an assessment of the necessity and proportionality of the processing operations in relation to the purposes, (c) an assessment of the risks to the rights and freedoms of data subjects, and (d) the measures envisaged to address the risks.
R10 - The processor shall not engage a sub-processor without a prior specific or general written authorization of the controller.
R11 - In case of general written authorization, the processor shall inform the controller of any intended changes concerning the addition or replacement of sub-processors.
R12 - The processor shall process personal data on documented instructions from the controller.
R13 - The processor can process personal data without documented instructions, if required by Union or Member State law.
R14 - The processor shall inform the controller of that legal requirement before processing, if law does not prohibit informing the controller on grounds of public interest.
R15 - The processor shall ensure that persons authorized to process personal data have committed themselves to confidentiality or are under an appropriate statutory obligation of confidentiality.
R16 - The processor shall take all measures required pursuant to Article 32 or to ensure the security of processing.
R17 - The processor shall assist the controller in fulfilling its obligation to respond to requests for exercising the data subject's rights.
R18 - The processor shall assist the controller in ensuring the security of processing.
R19 - The processor shall assist the controller in consulting the supervisory authorities prior to processing where the processing would result in a high risk in the absence of measures taken by the controller to mitigate the risk.
R20 - The processor shall assist the controller in notifying a personal data breach to the supervisory authority.
R21 - The processor shall assist the controller in communicating a personal data breach to the data subject.
R22 - The processor shall assist the controller in ensuring compliance with the obligations pursuant to data protection impact assessment (DPIA).
R23 - The processor shall return or delete all personal data to the controller after the end of the provision of services relating to processing.
R24 - The processor shall immediately inform the controller if an instruction infringes the GDPR or other data protection provisions.
R25 - The processor shall make available to the controller information necessary to demonstrate compliance with the obligations Article 28 in GDPR.
R26 - The processor shall allow for and contribute to audits, including inspections, conducted by the controller or another auditor mandated by the controller.
R27 - The processor shall impose the same obligations referred to in Article 28(3) in GDPR on the engaged sub-processors by way of contract or other legal act under Union or Member State law.
R28 - The processor shall remain fully liable to the controller for the performance of sub-processor's obligations.
R29 - When assessing the level of security, the processor shall take into account the risk of accidental or unlawful destruction, loss, alternation, unauthorized disclosure of or access to the personal data transmitted, stored or processed.
R30 - The processor shall not transfer personal data to a third country or international organization without  a prior specific or general authorization of the controller.
R31 - The processor can demonstrate guarantees to Article 28 (1--4) through adherence to an approved codes of conduct or an approved certification mechanism.
R32 - The processor shall implement appropriate technical and organisational measures to ensure a level of security appropriate to the risk of varying likelihood and severity for the rights and freedoms of natural persons.
R33 - The processor shall ensure that any natural person acting under its authority who has access to personal data only process them on instructions from the controller.
R34 - The processor shall notify the controller without undue delay after becoming aware of a personal data breach.
R35 - A processor shall be liable for the damage caused by processing only where it has not complied with obligations of the GDPR specifically directed to processors or where it has acted outside or contrary to lawful instructions of the controller.
R36 - In case of general written authorization, the controller shall have the right to object to changes concerning  the addition or replacement of sub-processors, after having been informed of such intended changes by the processor.
R37 - The controller shall have the right to suspend the processing in certain cases.
R38 - The controller shall have the right to terminate the DPA in certain cases.
R39 - The controller shall, no later than 72 hours after having become aware of it, notify the personal data breach to the supervisory authority.
R40 - The controller shall document the personal breaches.
R41 - In case of high risks, the controller shall communicate the data breach to the data subject without undue delay.
R42 - The controller shall carry out DPIA.
R43 - The controller shall seek advice of the DPO when carrying DPIA.
R44 - The controller shall seek the views of data subjects or their representatives on the intended processing.
R45 - The controller shall carry out a review to assess if processing is performed in accordance with the data protection impact assessment at least when there is a change of the risk represented by processing operations.
R46 - Any controller involved in processing shall be liable for the damage caused by processing which infringes the GDPR."""

In [67]:
import json

delimiter = "####"

# Convert sentences dataframe to a dictionary, grouped by Paragraph-id
sentences_grouped = {}
for _, row in df_sentences.iterrows():
    pid = row['Paragraph-id']
    if pid not in sentences_grouped:
        sentences_grouped[pid] = []
    sentences_grouped[pid].append((row['ID'], row['Target']))  # Storing sentence ID and Target

# Generate JSON data
data = []
for pid, sentence_info in sentences_grouped.items():
    # Merge sentences from the same paragraph with a delimiter
    user_message = delimiter.join([row['Sentence'] for _, row in df_sentences.iterrows() if row['Paragraph-id'] == pid])

    # Generate labels for each sentence
    assistant_message = ' '.join([f"{{text{sid}: R{target}}}" for sid, target in sentence_info])

    data.append({
        "messages": [
            {
                "role": "system",

                "content": f"You are a legal expert trained to identify applicable Compliance Rules based on a given text within its specific context. The list of Compliance Rules is as follows: {compliance_rules_text}. You are gonna be provided with text separated by #### that in total would make the context for the text at hand. When provided with the text and its context, your response should only include the rule identifier (e.g., text1: 'R5') if applicable. If there is no direct connection to any Compliance Rule within the context provided, respond with text1: 'R99'. Follow this format strictly."
            },
            {
                "role": "user",
                "content": f"{user_message}"
            },
            {
                "role": "assistant",
                "content": assistant_message
            }
        ]
    })

# Save to JSON file
with open('fine_tuning_data.json', 'w') as file:
    json.dump(data, file, indent=4)

print("JSON data generated and saved to fine_tuning_data.json")


JSON data generated and saved to fine_tuning_data.json


In [68]:
import numpy as np
# Count total tokens and assistant tokens
total_tokens = 0
assistant_tokens = 0
for item in data:
    messages = item['messages']
    total_tokens += num_tokens_from_messages(messages)
    assistant_tokens += num_assistant_tokens_from_messages(messages)

print(f"Total tokens in the dataset: {total_tokens}")
print(f"Total tokens generated by the assistant: {assistant_tokens}")

# If you want to see the distribution of token counts
token_counts = [num_tokens_from_messages(item['messages']) for item in data]
print_distribution(token_counts, "total tokens per item")

Total tokens in the dataset: 25473
Total tokens generated by the assistant: 769

#### Distribution of total tokens per item:
min / max: 1484, 2530
mean / median: 1698.2, 1624.0
p5 / p95: 1497.2, 1836.3999999999999


In [71]:

# Calculate tokens
total_tokens = sum(num_tokens_from_messages(item['messages']) for item in data)
assistant_tokens = sum(num_assistant_tokens_from_messages(item['messages']) for item in data)
system_tokens = sum(num_system_tokens_from_messages(item['messages']) for item in data)
user_tokens = sum(num_user_tokens_from_messages(item['messages']) for item in data)
total_system_and_user_tokens = system_tokens + user_tokens
print(f"Total tokens in the dataset: {total_tokens}")
print(f"Total assistant tokens in the dataset: {assistant_tokens}")
print(f"Total system tokens in the dataset: {system_tokens}")
print(f"Total user tokens in the dataset: {user_tokens}")
print(f"Total system and user tokens in the dataset: {total_system_and_user_tokens}")

Total tokens in the dataset: 25473
Total assistant tokens in the dataset: 769
Total system tokens in the dataset: 21585
Total user tokens in the dataset: 2894
Total system and user tokens in the dataset: 24479


zeroshot GPT-3.5-turbo




```
gpt-3.5-turbo-1106	$0.0010 / 1K tokens	$0.0020 / 1K tokens
```



zeroshot GPT-4-turbo

gpt-4-0125-preview	$0.01 / 1K tokens	$0.03 / 1K tokens

1 DPA

In [72]:
# GPT-3.5-turbo-1106 Pricing
gpt35_input_cost_per_1k_tokens = 0.0010
gpt35_output_cost_per_1k_tokens = 0.0020

# GPT-4-0125-preview Pricing
gpt4_input_cost_per_1k_tokens = 0.01
gpt4_output_cost_per_1k_tokens = 0.03

# Token counts
total_input_tokens = 24479  # Total system and user tokens
total_output_tokens = 769  # Total assistant tokens
total_tokens = 25473  # All tokens, considering fine-tuning where every token is input

# Calculating costs
gpt35_input_cost = (total_input_tokens / 1000) * gpt35_input_cost_per_1k_tokens
gpt35_output_cost = (total_output_tokens / 1000) * gpt35_output_cost_per_1k_tokens
gpt35_total_cost = gpt35_input_cost + gpt35_output_cost

gpt4_input_cost = (total_input_tokens / 1000) * gpt4_input_cost_per_1k_tokens
gpt4_output_cost = (total_output_tokens / 1000) * gpt4_output_cost_per_1k_tokens
gpt4_total_cost = gpt4_input_cost + gpt4_output_cost

# Fine-tuning costs (where every token is considered input)
gpt35_finetuning_cost = (total_tokens / 1000) * gpt35_input_cost_per_1k_tokens
gpt4_finetuning_cost = (total_tokens / 1000) * gpt4_input_cost_per_1k_tokens

# Outputting costs
gpt35_costs = (gpt35_input_cost, gpt35_output_cost, gpt35_total_cost)
gpt4_costs = (gpt4_input_cost, gpt4_output_cost, gpt4_total_cost)
finetuning_costs = (gpt35_finetuning_cost, gpt4_finetuning_cost)

gpt35_costs, gpt4_costs, finetuning_costs


((0.024479, 0.001538, 0.026017000000000002),
 (0.24479, 0.02307, 0.26786),
 (0.025473, 0.25473))

####For GPT-3.5-turbo-1106:

####Input cost: $0.02448

####Output cost: $0.00154

####Total cost: $0.02602

####For GPT-4-0125-preview:

####Input cost: $0.24479

####Output cost: $0.02307

####Total cost: $0.26786

####For fine-tuning (where all tokens are considered input):

####GPT-3.5-turbo-1106: $0.02547

####GPT-4-0125-preview: $0.25473